In [ ]:
## --- PARAMETER ---
CONTEST = "W407" # Contest name
CONTEST_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-407" # API URL

PAGE = 470 # how many page gonna fetch 

headers1 = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version': '"126.0.6478.127"',
    'sec-ch-ua-full-version-list': '"Not/A)Brand";v="8.0.0.0", "Chromium";v="126.0.6478.127", "Google Chrome";v="126.0.6478.127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"15.0.0"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

headers2 = headers1.copy()
headers2['referer'] = 'https://leetcode.com/contest/'

In [ ]:
import concurrent.futures
import csv
import glob
import json
import os
import random
import sys
from tqdm import tqdm
import urllib.error
import urllib.request

# create folder
try:
    os.mkdir(CONTEST)
except:
    pass
try:
    os.mkdir(CONTEST+"/Q3")
except:
    pass
try:
    os.mkdir(CONTEST+"/Q4")
except:
    pass
for i in range(12):
    try:
        os.mkdir(CONTEST+"/Q3/"+str(i)+"000-"+str(i)+"999")
    except:
        pass
    try:
        os.mkdir(CONTEST+"/Q4/"+str(i)+"000-"+str(i)+"999")
    except:
        pass

In [ ]:
## get question_id
Q = []

status = 0
for _ in range(10):
    try:
        req = urllib.request.Request(CONTEST_URL, headers=random.choice([headers1, headers2]))
        r = urllib.request.urlopen(req)
        response = json.loads(r.read())
    except urllib.error.HTTPError as e:
        status = e.code
    except:
        pass
    else:
        break
else:
    print("Status", status, CONTEST_URL)
    sys.exit()
    
for question in response["questions"]:
    Q.append(str(question["question_id"]))
    print(question["question_id"], question["credit"], question["title"])

In [ ]:
## create the csv to record user name and submission_id
with open(CONTEST+"/"+CONTEST+".csv", "w") as f:
    f.write("user, Q1, Q2, Q3, Q4\n");
    
    for pagination in tqdm(range(1, PAGE+1)):
        for _ in range(10):
            try:
                req = urllib.request.Request(CONTEST_URL+"?pagination=" + str(pagination) +"&region=global", headers=random.choice([headers1, headers2]))
                r = urllib.request.urlopen(req)
                response = json.loads(r.read())
            except:
                pass
            else:
                break
        else:
            print("http request error:", CONTEST_URL+"?pagination=" + str(pagination) +"&region=global")
            print(r.status)
            sys.exit()

        for i in range(25):            
            f.write(response["total_rank"][i]["user_slug"])
            
            for j, Q_id in enumerate(Q):
                if Q_id not in response["submissions"][i]:
                    submission_id = ""
                else:   
                    submission_id = str(response["submissions"][i][Q_id]["submission_id"])
    
                f.write("," + submission_id)
            f.write("," + response["total_rank"][i]["data_region"])

            f.write("\n")

In [ ]:
## --- PARAMETER ---
## download code
Q = 3 # i.e. 4 means Q4

def lang_ext(lang):
    match lang:
        case "cpp":
            return "cpp"
        case "python3" | "python":
            return "py"
        case "java":
            return "java"
        case _:
            return lang
def downloadCode(rows):
    rank, row = rows
    if rank == 0 or row[5] == 'CN':
        return
        
    folder = '/'+str(rank//1000)+"000-"+str(rank//1000)+"999/"
    
    if row[Q] and not glob.glob(CONTEST+"/Q"+str(Q)+folder+str(rank)+".*"):
        submission_id = row[Q]
        
        for _ in range(20):
            try:
                req = urllib.request.Request("https://leetcode.com/api/submissions/" + submission_id, headers=random.choice([headers1, headers2]))
                r = urllib.request.urlopen(req)
                response = json.loads(r.read())
            except:
                pass
            else:
                with open(CONTEST+"/Q"+str(Q)+folder+str(rank)+"."+lang_ext(response["lang"]), "w", encoding="utf-8") as f2:
                    try:
                        f2.write(response["code"])
                    except:
                        print(submission_id)
                        sys.exit()
                break
        else:
            print("file missed", rank, submission_id)

with open(CONTEST+"/"+CONTEST+".csv") as f:
    rows = list(csv.reader(f))
with concurrent.futures.ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(downloadCode, enumerate(rows)), total=len(rows)))

print("finished!!!")